# **Entregable Semana 4**
**📄 Documentación del Proceso:**

**🧩 Selección de Tipos de Gráficas**

Se eligieron cuatro tipos de gráficas con base en la naturaleza de los datos meteorológicos y su propósito informativo:

Gráfico de líneas: ideal para observar la evolución de la temperatura en distintos días. Muestra tendencias y fluctuaciones.

Gráfico de barras: permite comparar la humedad entre diferentes fechas, facilitando la comparación visual.

Diagrama de dispersión: utilizado para analizar la relación entre temperatura y presión. Detecta correlaciones o patrones atípicos.

**🧠 Razonamiento y Metodología**
Se consideraron las variables más relevantes para el análisis climático: temperatura, humedad y presión atmosférica.

Cada gráfica se diseñó para responder preguntas específicas del problema, como:

¿Cómo varía la temperatura a lo largo de los días?

¿Qué tan uniforme es la humedad?

¿Existe relación entre temperatura y presión?

¿Qué proporción de datos corresponde a pronósticos?

**📊 Contribución de Cada Gráfica**

Líneas: permite identificar rápidamente tendencias o anomalías.

Barras: destaca los días con mayor o menor humedad.

Dispersión: revela posibles relaciones físicas entre variables.

**🧾 Almacenamiento de Datos en Excel**

**🛠️ Herramientas Utilizadas**

Se empleó el módulo openpyxl para exportar datos validados y analizados a un archivo .xlsx.

**🗂️ Organización de la Información**

-Fecha

-Ciudad

-Temperatura

-Humedad

-Presión

-Tipo (Clima actuál o pronóstico)

**📑 Estructura del Libro de Excel**

Se utilizó una estructura en formato de tabla.

Las columnas tienen filtros activados para facilitar la búsqueda y comparación.

**📈 Diseño de Gráficas**

**📊 Tipos de Gráficas Generadas**

**Gráfico de Líneas**

Uso: temperatura por fecha.

Detalles: eje X con fechas, eje Y con °C, título: “Temperatura Diaria”.

**Gráfico de Barras**

Uso: humedad relativa por día.

Detalles: ejes con etiquetas y %.

**Diagrama de Dispersión**

Uso: relación entre temperatura y presión.

Detalles: cada punto representa un registro, útil para inferir patrones.


In [ ]:
!pip install requests
!pip install plotext
!pip install openpyxl

In [ ]:
import requests
import csv
import json
from datetime import datetime
import statistics
import plotext as plt
from openpyxl import Workbook

API_KEY = "f0f0c07c117721896152aeca0fbbe31a"

def get_weather_current(city):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    res = requests.get(url)
    if res.status_code != 200:
        print(f"❌ Error: Ciudad '{city}' no encontrada o API falló.")
        return None
    data = res.json()
    return {
        "fecha": datetime.utcfromtimestamp(data["dt"]).strftime("%Y-%m-%d"),
        "ciudad": data["name"],
        "temperatura": data["main"]["temp"],
        "humedad": data["main"]["humidity"],
        "presion": data["main"]["pressure"],
        "tipo": "actual"
    }

def get_weather_forecast(city):
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
    res = requests.get(url)
    if res.status_code != 200:
        print(f"❌ Error: Ciudad '{city}' no encontrada o API falló.")
        return []
    data = res.json()
    forecast_data = []
    for item in data["list"]:
        dt_txt = item["dt_txt"]
        if dt_txt.endswith("18:00:00"):
            forecast_data.append({
                "fecha": dt_txt.split()[0],
                "ciudad": data["city"]["name"],
                "temperatura": item["main"]["temp"],
                "humedad": item["main"]["humidity"],
                "presion": item["main"]["pressure"],
                "tipo": "pronostico"
            })
    return forecast_data

def guardar_csv(datos, nombre_archivo):
    campos = ["fecha", "ciudad", "temperatura", "humedad", "presion", "tipo"]
    with open(nombre_archivo, mode='w', encoding='utf-8', newline='') as archivo:
        escritor = csv.DictWriter(archivo, fieldnames=campos)
        escritor.writeheader()
        escritor.writerows(datos)

def guardar_json(datos, nombre_archivo):
    with open(nombre_archivo, mode='w', encoding='utf-8') as archivo:
        json.dump(datos, archivo, indent=4, ensure_ascii=False)

def guardar_excel(datos, nombre_archivo):
    wb = Workbook()
    ws = wb.active
    ws.title = "Datos Clima"
    ws.append(["Fecha", "Ciudad", "Temperatura (°C)", "Humedad (%)", "Presión (hPa)", "Tipo"])
    for d in datos:
        ws.append([d["fecha"], d["ciudad"], d["temperatura"], d["humedad"], d["presion"], d["tipo"]])
    wb.save(nombre_archivo)

def leer_datos_csv(nombre_archivo):
    datos = []
    try:
        with open(nombre_archivo, mode='r', encoding='utf-8') as archivo:
            lector = csv.DictReader(archivo)
            for fila in lector:
                datos.append({
                    "fecha": fila["fecha"],
                    "ciudad": fila["ciudad"],
                    "temperatura": float(fila["temperatura"]),
                    "humedad": float(fila["humedad"]),
                    "presion": float(fila["presion"]),
                    "tipo": fila["tipo"]
                })
    except FileNotFoundError:
        print(f"❌ Archivo CSV no encontrado: {nombre_archivo}")
    except Exception as e:
        print(f"❌ Error leyendo CSV: {e}")
    return datos

def analizar_datos(datos):
    if not datos:
        print("⚠️ No hay datos para analizar.")
        return

    temperaturas = [d["temperatura"] for d in datos]
    humedades = [d["humedad"] for d in datos]
    presiones = [d["presion"] for d in datos]

    print("\n📈 Resultados del análisis:")
    print(f"🌡️ Temperatura promedio: {statistics.mean(temperaturas):.2f} °C")
    if len(temperaturas) > 1:
        print(f"📉 Desviación estándar de la temperatura: {statistics.stdev(temperaturas):.2f} °C")
    else:
        print("⚠️ No hay suficientes datos para calcular la desviación estándar.")
    print(f"💧 Humedad promedio: {statistics.mean(humedades):.2f} %")
    print(f"🧭 Rango de presión atmosférica: {min(presiones):.2f} - {max(presiones):.2f} hPa")
    ciudades = set(d["ciudad"] for d in datos)
    fechas = set(d["fecha"] for d in datos)
    print(f"🏙️ Ciudades analizadas: {', '.join(ciudades)}")
    print(f"📅 Días analizados: {', '.join(fechas)}")

from datetime import datetime

def graficar_lineas(datos):
    import plotext as plt

    fechas = sorted(set(d["fecha"] for d in datos))
    fechas_convertidas = [datetime.strptime(f, "%Y-%m-%d").strftime("%d/%m/%Y") for f in fechas]

    temperaturas = []
    for fecha in fechas:
        temps = [d["temperatura"] for d in datos if d["fecha"] == fecha]
        temperaturas.append(sum(temps)/len(temps))

    plt.clear_figure()
    plt.title("🌡️ Temperatura promedio diaria")
    plt.plot(fechas_convertidas, temperaturas, marker='dot')
    plt.xlabel("Fecha (DD/MM/YYYY)")
    plt.ylabel("Temperatura (°C)")
    plt.grid(True)
    plt.show()

def graficar_barras(datos):
    import plotext as plt

    fechas = sorted(set(d["fecha"] for d in datos))
    fechas_convertidas = [datetime.strptime(f, "%Y-%m-%d").strftime("%d/%m/%Y") for f in fechas]

    humedades = []
    for fecha in fechas:
        hums = [d["humedad"] for d in datos if d["fecha"] == fecha]
        humedades.append(sum(hums)/len(hums))

    plt.clear_figure()
    plt.title("💧 Humedad promedio diaria")
    plt.bar(fechas_convertidas, humedades, color="cyan")
    plt.xlabel("Fecha (DD/MM/YYYY)")
    plt.ylabel("Humedad (%)")
    plt.grid(True)
    plt.show()

def graficar_dispersion(datos):
    import plotext as plt

    temperaturas = [d["temperatura"] for d in datos]
    presiones = [d["presion"] for d in datos]

    plt.clear_figure()
    plt.title("🧭 Presión vs Temperatura")
    plt.scatter(temperaturas, presiones)
    plt.xlabel("Temperatura (°C)")
    plt.ylabel("Presión (hPa)")
    plt.grid(True)
    plt.show()


def menu_graficas(datos):
    while True:
        print("\n📊 Elige una opción para visualizar:")
        print("1. Gráfico de líneas (Temperatura)")
        print("2. Gráfico de barras (Humedad)")
        print("3. Diagrama de dispersión (Presión vs Temperatura)")
        print("4. Ver todas")
        print("5. Salir")

        opcion = input("👉 Tu elección: ").strip()
        if opcion == "1":
            graficar_lineas(datos)
        elif opcion == "2":
            graficar_barras(datos)
        elif opcion == "3":
            graficar_dispersion(datos)
        elif opcion == "4":
            graficar_lineas(datos)
            graficar_barras(datos)
            graficar_dispersion(datos)
        elif opcion == "5":
            break
        else:
            print("⚠️ Opción inválida. Intenta de nuevo.")

def mostrar_datos(actual, pronostico):
    print(f"\n🌆 Clima actual en {actual['ciudad']} ({actual['fecha']}):")
    print(f"  Temperatura: {actual['temperatura']} °C")
    print(f"  Humedad: {actual['humedad']} %")
    print(f"  Presión: {actual['presion']} hPa")
    print("\n📅 Pronóstico a 5 días (18:00 hrs):")
    for p in pronostico:
        print(f"  {p['fecha']} -> Temp: {p['temperatura']} °C, Humedad: {p['humedad']} %, Presión: {p['presion']} hPa")

def main():
    ciudad = input("🌆 Ingrese la ciudad para consultar el clima: ").strip()
    if not ciudad:
        print("❌ No ingresó ninguna ciudad.")
        return

    actual = get_weather_current(ciudad)
    if actual is None:
        return

    pronostico = get_weather_forecast(ciudad)

    mostrar_datos(actual, pronostico)

    datos_guardar = [actual] + pronostico

    guardar_csv(datos_guardar, "clima_datos.csv")
    guardar_json(datos_guardar, "clima_datos.json")
    guardar_excel(datos_guardar, "clima_datos.xlsx")

    print(f"\n✅ Datos guardados en 'clima_datos.csv', 'clima_datos.json' y 'clima_datos.xlsx'")

    datos_csv = leer_datos_csv("clima_datos.csv")

    print("\n--- Análisis basado en CSV ---")
    analizar_datos(datos_csv)

    menu_graficas(datos_csv)

if __name__ == "__main__":
    main()


KeyboardInterrupt: Interrupted by user